In [1]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

os.environ['WANDB_DISABLED'] = "true"

In [2]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo
# !gdown 1mq9CWT9MXsH1Y-ihxVL4uEzufQjpBoer
!gdown 1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql

Downloading...
From: https://drive.google.com/uc?id=1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql
To: /content/processed_data.zip
100% 6.00M/6.00M [00:00<00:00, 100MB/s]


In [3]:
!rm -rf processed_data

!unzip -q processed_data.zip

In [4]:
!pip install -q transformers[torch] accelerate -U

!pip install -q datasets

!pip install -q transformers

!pip install -q iterative-stratification

!pip install -q auto-gptq optimum bitsandbytes

In [5]:
import numpy as np

import pandas as pd

import os

from os import path

import glob

from tqdm.notebook import tqdm

from tqdm import tqdm

import string

import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy,BitsAndBytesConfig

from datasets import Dataset,load_dataset, DatasetDict

# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import classification_report,accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support

from sklearn.utils import shuffle

from sklearn.preprocessing import LabelEncoder

from scipy.special import expit

from peft import prepare_model_for_kbit_training

from peft import LoraConfig, get_peft_model,PeftModel


In [6]:
import torch

import torch.nn as nn

import torch.nn.functional as F

In [131]:


models_list_3=[
        # "FacebookAI/xlm-roberta-large",
      # "google-bert/bert-base-uncased",
        "rufimelo/bert-large-portuguese-cased-sts",

#      "google-bert/bert-base-multilingual-uncased",

          ]



models_name_3=[
# "FacebookAI/xlm-roberta-large",
# "google-bert/bert-base-uncased",
        "rufimelo/bert-large-portuguese-cased-sts",

# "bert",

]


In [132]:
kind="train"

task="track_a"

# langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]
# labels=['Anger','Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
# langs=["amh","arq","ary","chn","deu","esp","hau","hin","ibo","kin","mar","orm","pcm","ptbr","ptmz","ron","rus","som","sun","swa","swe","tat","tir","ukr","vmw","yor","ind","jav","xho","zul"]
langs=["ptbr","ptmz"]
# langs=["afr"]


# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"



# train_data=pd.read_csv(processed_path)


In [133]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)




In [134]:
all_csv

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,ptbr_train_track_a_00001,0,0,0,0,0,0,ptbr,moça eu fiz uma análise com o tinder uma vez e...
1,ptbr_train_track_a_00002,1,1,0,0,0,0,ptbr,eles sempre mostram a míseria em como tudo era...
2,ptbr_train_track_a_00003,0,0,0,1,1,0,ptbr,eu nunca quis tanto algo como quero você jamai...
3,ptbr_train_track_a_00004,1,0,0,0,0,0,ptbr,esperando aqui o stf vim me emparedar do eu ex...
4,ptbr_train_track_a_00005,1,0,0,0,0,0,ptbr,e no final o ps vai pagar o pato ja é o 4° tre...
...,...,...,...,...,...,...,...,...,...
3767,ptmz_train_track_a_01542,0,0,0,1,0,0,ptmz,salários mínimos vão aumentar em moçambique
3768,ptmz_train_track_a_01543,0,0,0,1,0,0,ptmz,covid19 organizações da sociedade civil engaja...
3769,ptmz_train_track_a_01544,0,0,0,0,0,0,ptmz,friagem atinge a amazônia peruana consequência...
3770,ptmz_train_track_a_01545,0,0,0,0,0,0,ptmz,can 2021 favoritos ganham por vantagem mínima ...


In [135]:
all_csv[all_csv["lang"]=="eng"]

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message


In [136]:
# float_columns = all_csv.select_dtypes(include=['float64']).columns
# # Check each column and convert only if there are no NaN values
# for col in float_columns:
#     if all_csv[col].isna().sum() == 0:  # No NaN values in the column
#         print(f"converting {kind} {task} {lang} column {col} to int")
#         all_csv[col] = all_csv[col].astype('int')
#     else:
#         print(f"converting {kind} {task} {lang} column {col} to int")

In [137]:
train_data=all_csv


In [138]:
# train_data = train_data.drop(columns=['disgust'],axis=1)


In [139]:

train_data=train_data.fillna(0)

train_data

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,ptbr_train_track_a_00001,0,0,0,0,0,0,ptbr,moça eu fiz uma análise com o tinder uma vez e...
1,ptbr_train_track_a_00002,1,1,0,0,0,0,ptbr,eles sempre mostram a míseria em como tudo era...
2,ptbr_train_track_a_00003,0,0,0,1,1,0,ptbr,eu nunca quis tanto algo como quero você jamai...
3,ptbr_train_track_a_00004,1,0,0,0,0,0,ptbr,esperando aqui o stf vim me emparedar do eu ex...
4,ptbr_train_track_a_00005,1,0,0,0,0,0,ptbr,e no final o ps vai pagar o pato ja é o 4° tre...
...,...,...,...,...,...,...,...,...,...
3767,ptmz_train_track_a_01542,0,0,0,1,0,0,ptmz,salários mínimos vão aumentar em moçambique
3768,ptmz_train_track_a_01543,0,0,0,1,0,0,ptmz,covid19 organizações da sociedade civil engaja...
3769,ptmz_train_track_a_01544,0,0,0,0,0,0,ptmz,friagem atinge a amazônia peruana consequência...
3770,ptmz_train_track_a_01545,0,0,0,0,0,0,ptmz,can 2021 favoritos ganham por vantagem mínima ...


In [140]:
float_columns = train_data.select_dtypes(include=['float64']).columns

# Convert those columns to int
train_data[float_columns] = train_data[float_columns].astype('int')
train_data

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,ptbr_train_track_a_00001,0,0,0,0,0,0,ptbr,moça eu fiz uma análise com o tinder uma vez e...
1,ptbr_train_track_a_00002,1,1,0,0,0,0,ptbr,eles sempre mostram a míseria em como tudo era...
2,ptbr_train_track_a_00003,0,0,0,1,1,0,ptbr,eu nunca quis tanto algo como quero você jamai...
3,ptbr_train_track_a_00004,1,0,0,0,0,0,ptbr,esperando aqui o stf vim me emparedar do eu ex...
4,ptbr_train_track_a_00005,1,0,0,0,0,0,ptbr,e no final o ps vai pagar o pato ja é o 4° tre...
...,...,...,...,...,...,...,...,...,...
3767,ptmz_train_track_a_01542,0,0,0,1,0,0,ptmz,salários mínimos vão aumentar em moçambique
3768,ptmz_train_track_a_01543,0,0,0,1,0,0,ptmz,covid19 organizações da sociedade civil engaja...
3769,ptmz_train_track_a_01544,0,0,0,0,0,0,ptmz,friagem atinge a amazônia peruana consequência...
3770,ptmz_train_track_a_01545,0,0,0,0,0,0,ptmz,can 2021 favoritos ganham por vantagem mínima ...


In [141]:
train_data.dtypes

,0
id,object
anger,int64
disgust,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
lang,object
clean_message,object


In [142]:
train_data = train_data[

train_data['clean_message'].isnull()==False]

train_data.reset_index()

,index,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,0,ptbr_train_track_a_00001,0,0,0,0,0,0,ptbr,moça eu fiz uma análise com o tinder uma vez e...
1,1,ptbr_train_track_a_00002,1,1,0,0,0,0,ptbr,eles sempre mostram a míseria em como tudo era...
2,2,ptbr_train_track_a_00003,0,0,0,1,1,0,ptbr,eu nunca quis tanto algo como quero você jamai...
3,3,ptbr_train_track_a_00004,1,0,0,0,0,0,ptbr,esperando aqui o stf vim me emparedar do eu ex...
4,4,ptbr_train_track_a_00005,1,0,0,0,0,0,ptbr,e no final o ps vai pagar o pato ja é o 4° tre...
...,...,...,...,...,...,...,...,...,...,...
3767,3767,ptmz_train_track_a_01542,0,0,0,1,0,0,ptmz,salários mínimos vão aumentar em moçambique
3768,3768,ptmz_train_track_a_01543,0,0,0,1,0,0,ptmz,covid19 organizações da sociedade civil engaja...
3769,3769,ptmz_train_track_a_01544,0,0,0,0,0,0,ptmz,friagem atinge a amazônia peruana consequência...
3770,3770,ptmz_train_track_a_01545,0,0,0,0,0,0,ptmz,can 2021 favoritos ganham por vantagem mínima ...


In [143]:
train_data

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,ptbr_train_track_a_00001,0,0,0,0,0,0,ptbr,moça eu fiz uma análise com o tinder uma vez e...
1,ptbr_train_track_a_00002,1,1,0,0,0,0,ptbr,eles sempre mostram a míseria em como tudo era...
2,ptbr_train_track_a_00003,0,0,0,1,1,0,ptbr,eu nunca quis tanto algo como quero você jamai...
3,ptbr_train_track_a_00004,1,0,0,0,0,0,ptbr,esperando aqui o stf vim me emparedar do eu ex...
4,ptbr_train_track_a_00005,1,0,0,0,0,0,ptbr,e no final o ps vai pagar o pato ja é o 4° tre...
...,...,...,...,...,...,...,...,...,...
3767,ptmz_train_track_a_01542,0,0,0,1,0,0,ptmz,salários mínimos vão aumentar em moçambique
3768,ptmz_train_track_a_01543,0,0,0,1,0,0,ptmz,covid19 organizações da sociedade civil engaja...
3769,ptmz_train_track_a_01544,0,0,0,0,0,0,ptmz,friagem atinge a amazônia peruana consequência...
3770,ptmz_train_track_a_01545,0,0,0,0,0,0,ptmz,can 2021 favoritos ganham por vantagem mínima ...


In [144]:
# label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']
label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']


In [145]:
train_data['lang'].value_counts()

,count
lang,
ptbr,2226
ptmz,1546


In [146]:
sum(train_data['lang'].isna())

0

In [147]:
nan_rows = train_data[train_data.isna().any(axis=1)]
nan_rows
# Group by 'lang' and list the rows with NaNs for each language
# nan_by_language = nan_rows.groupby('lang').apply(lambda x: x)

# Display the result
# print(nan_by_language)

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message


In [148]:
nan_rows[nan_rows["lang"]=="eng"]

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message


In [149]:
train_data[train_data["lang"]=="deu"].dtypes

,0
id,object
anger,int64
disgust,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
lang,object
clean_message,object


In [150]:
nan_rows["lang"].value_counts()

,count
lang,


In [151]:
nan_rows.dtypes

,0
id,object
anger,int64
disgust,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
lang,object
clean_message,object


In [152]:
# Step 2: Count how many columns are not 0 or 1 for each language
def count_nonzero_nonone_rows(group):
    # Exclude the 'lang' column from counting if needed
    # Use boolean conditions to count values that are not 0 or 1
    return (group.drop(columns='anger') != 0) & (group.drop(columns='anger') != 1).sum(axis=1).sum()

# Group by 'lang' and apply the function to count non-zero, non-one values
non_zero_non_one_counts = train_data.groupby('lang').apply(count_nonzero_nonone_rows)

# Display the results
# result = pd.DataFrame({
#     'row_count': lang_row_counts,
#     'non_zero_non_one_count': non_zero_non_one_counts
# })

print(non_zero_non_one_counts)

              id  disgust   fear    joy  sadness  surprise   lang  \
lang                                                                
ptbr 0     False    False  False  False    False     False  False   
     1     False    False  False  False    False     False  False   
     2     False    False  False  False    False     False  False   
     3     False    False  False  False    False     False  False   
     4     False    False  False  False    False     False  False   
...          ...      ...    ...    ...      ...       ...    ...   
ptmz 3767  False    False  False  False    False     False  False   
     3768  False    False  False  False    False     False  False   
     3769  False    False  False  False    False     False  False   
     3770  False    False  False  False    False     False  False   
     3771  False    False  False  False    False     False  False   

           clean_message  
lang                      
ptbr 0             False  
     1             Fa

<ipython-input-152-06fbda4bf1b6>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  non_zero_non_one_counts = train_data.groupby('lang').apply(count_nonzero_nonone_rows)


In [153]:
maxi=train_data['clean_message'].apply(len).max()
maxi

2576

In [154]:
train_data[train_data['clean_message'].apply(len) == maxi]

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
472,ptbr_train_track_a_00473,0,0,0,0,0,0,ptbr,o grande diferencial de um colégio não é só o ...


In [155]:
train_data[train_data['clean_message'].apply(len) == maxi]["clean_message"].values

array(['o grande diferencial de um colégio não é só o ensino mas a qualidade dos alunos que é o que vai te passar uma noção de média melhor por exemplo em um colégio excelente a média vai ser muito mas muito mais alta entre os alunos ah sim agora eu entendi o que você quis dizer sim talvez esse realmente seja o melhor dos casos mas eu fui fazer o vestibulinho pra provas de colégios desse estilo muito cedo grande parte da matéria eu acabava sem saber muito sobre e tals então atrapalha o ensino é certamente ótimo no colégio onde fui e muitos dos alunos vão bem em vestibular facilmente mas as atividades olímpicas estiveram falhando ao menos na época em que estudei também tem que mesmo sendo um ótimo colégio e renomado por ser particular e ter somente uma unidade não tem a mesma segregação que colégios objetivo ou etapa possuem a única unidade tem que englobar todos os tipos de alunos que é diferente dos exemplos onde os alunos vão para diferentes unidades praticamente baseado em seu nível

In [156]:
# shuffle data
train_data=train_data.sample(frac=1)
# train_data=train_data.sample(100,random_state=42)


In [157]:
# train valid split
train,valid=train_test_split(train_data,test_size=0.2,random_state=42)

In [158]:
x_train, y_train = train['clean_message'].values.tolist(), train[label_columns].values.tolist()

x_valid, y_valid = valid['clean_message'].values.tolist(), valid[label_columns].values.tolist()

In [159]:
y_train_df = pd.DataFrame(y_train, columns=label_columns)

In [160]:
#update label column name

label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)

label_distribution.columns = ['count_0', 'count_1']
label_distribution['sum'] = label_distribution['count_0'] + label_distribution['count_1']
print(label_distribution)

          count_0  count_1   sum
anger        2355      662  3017
disgust      2910      107  3017
fear         2855      162  3017
joy          2373      644  3017
sadness      2534      483  3017
surprise     2821      196  3017


In [161]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [162]:
from sklearn.utils.class_weight import compute_class_weight

counts_0 = label_distribution['count_0'].to_numpy()
counts_1 = label_distribution['count_1'].to_numpy()

# Compute class-specific weights for each label (each class)
class_weights = []
for i in range(len(counts_0)):
    weight = compute_class_weight('balanced', classes=np.array([0, 1]), y=[0] * counts_0[i] + [1] * counts_1[i])
    class_weights.append(weight)

# Convert the list of weights to a tensor
class_weights_tensor = torch.tensor([w[1] for w in class_weights], dtype=torch.float,device=device)
class_weights_tensor

tensor([ 2.2787, 14.0981,  9.3117,  2.3424,  3.1232,  7.6964], device='cuda:0')

In [163]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, encodings, labels):

        self.encodings = encodings

        self.labels = labels



    def __getitem__(self, idx):

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)

        return item


    def __len__(self):

        return len(self.labels)

In [164]:
!ls output/best-model-v1

ls: cannot access 'output/best-model-v1': No such file or directory


In [165]:
def build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH):

  train_encodings = tokenizer(x_train, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  valid_encodings = tokenizer(x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  train_dataset = Dataset(train_encodings, y_train)

  valid_dataset = Dataset(valid_encodings, y_valid)

  return train_dataset,valid_dataset



def train_arg():

    training_args = TrainingArguments(

    output_dir="./output/best-model-v1",

    report_to=None,
    dataloader_pin_memory=False,

    num_train_epochs=3,

    gradient_accumulation_steps=2,

    per_device_train_batch_size=8,

    per_device_eval_batch_size=16,

    learning_rate=2e-5,

    weight_decay=0.01,

    eval_strategy=IntervalStrategy.STEPS,

    save_total_limit=1,

    logging_dir="./logs",

    logging_strategy=IntervalStrategy.STEPS,

    logging_steps=250,

    eval_steps=500,

    load_best_model_at_end=True,

    lr_scheduler_type="cosine_with_restarts",

    warmup_steps=500,

    save_strategy="steps",

    save_steps=500,

  )

    return training_args

In [166]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(eval_pred):
    """
    Compute metrics for multi-label classification.

    Args:
        eval_pred: Tuple of (predictions, labels).
            predictions: logits or probabilities of shape (batch_size, num_labels)
            labels: ground truth of shape (batch_size, num_labels)

    Returns:
        metrics: Dictionary containing accuracy, precision, recall, and F1-score.
    """
    # Unpack predictions and labels
    logits, labels = eval_pred

    # Convert logits to binary predictions
    # If your model outputs probabilities (e.g., after sigmoid), use threshold=0.5
    probabilities = expit(logits)
    predictions = (probabilities > 0.5).astype(int)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=None
    )

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [167]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from scipy.special import expit  # For sigmoid activation if needed

def evaluate_model(model, valid_dataset, compute_metrics, batch_size=32, device="cuda",average=None):
    """
    Evaluate a multi-label classification model without using the Hugging Face Trainer.

    Args:
        model: The trained model (Hugging Face model).
        valid_dataset: The validation dataset (torch.utils.data.Dataset).
        compute_metrics: The function to compute evaluation metrics.
        batch_size: Batch size for evaluation.
        device: Device to perform evaluation (e.g., "cuda" or "cpu").

    Returns:
        metrics: Dictionary containing evaluation metrics.
    """
    # Set model to evaluation mode
    model.eval()
    model.to(device)

    # Create DataLoader for the validation dataset
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    all_predictions = []
    all_labels = []

    # Disable gradient computation for evaluation
    with torch.no_grad():
        for batch in tqdm(valid_loader, desc="Evaluating"):
            # Get inputs and labels
            inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
            labels = batch["labels"].to(device)

            # Forward pass: Get logits
            logits = model(**inputs)

            # Apply sigmoid activation to logits if necessary (for multi-label classification)
            probabilities = expit(logits.cpu().numpy())  # Convert logits to probabilities
            predictions = (probabilities > 0.5).astype(int)  # Convert probabilities to binary predictions

            # Collect predictions and labels
            all_predictions.append(predictions)
            all_labels.append(labels.cpu().numpy())
    print()
    # Concatenate all predictions and labels
    all_predictions = np.concatenate(all_predictions, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Unpack predictions and labels
    logits, labels = all_predictions,all_labels

    # Convert logits to binary predictions
    # If your model outputs probabilities (e.g., after sigmoid), use threshold=0.5
    probabilities = expit(logits)
    predictions = (probabilities > 0.5).astype(int)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=average
    )
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    },all_predictions

In [168]:
def evaluate_func(trainer):
    metrics = evaluate_model(
    model=model,
    valid_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    batch_size=32,
    device=device
)
    accuracy=metrics["accuracy"]
    precision=metrics["precision"]
    recall=metrics["recall"]
    f1=metrics["f1"]

    return precision, recall, f1, accuracy

In [169]:
MAX_LENGTH = 512

metrics_df = pd.DataFrame(columns=["Models_name", "Precision", "Recall", "F1_score", "Accuracy"])

confusion_matrices = []

In [170]:
quantization_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_use_double_quant=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

)

In [171]:
models_list=models_list_3

models_name=models_name_3

x=models_list[0]

In [172]:
token=""

import huggingface_hub

if "gemma" in x or "Phi" in x:

  huggingface_hub.login(token=token)

In [173]:
USE_LORA=False

In [174]:
from transformers import AutoModel
from sentence_transformers import SentenceTransformer

In [175]:


torch.cuda.empty_cache()



print(x)
use_gpu = torch.cuda.is_available()
embedding_model = SentenceTransformer(
        x,
        trust_remote_code=True,
        device="cpu" if not use_gpu else "cuda",
        model_kwargs={
            "torch_dtype": torch.bfloat16 if use_gpu else torch.float32,
            # "attn_implementation": "sdpa"
        },
        # vector_dim must be 12288, 1024, 512, 256
        ## 1024 is recommended
        # set is_text_encoder 'True', if you do not encode image
        # config_kwargs={"is_text_encoder": True, "vector_dim": 1024},
    )
    # We can reduce the max_seq_length from the default of 2048 for faster encoding
embedding_model.max_seq_length = MAX_LENGTH

rufimelo/bert-large-portuguese-cased-sts


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.14k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/678k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Invalid model-index. Not loading eval results into CardData.


1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Invalid model-index. Not loading eval results into CardData.


In [176]:
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()

    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear4bit" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]

            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)
find_target_modules(embedding_model)

[]

In [177]:
# a=

In [178]:
embedding_model.eval() # embedding_model in training mode (dropout modules are activated)

if USE_LORA:

  # enable gradient check pointing

  embedding_model.gradient_checkpointing_enable()

  target_modules_dict={

      "microsoft/Phi-3.5-mini-instruct":'all-linear',
      "unsloth/Llama-3.2-1B-Instruct":["q_proj","k_proj","v_proj"],
      "unsloth/Qwen2.5-3B-Instruct-bnb-4bit":["q_proj","k_proj","v_proj"],
  }

  target_modules= target_modules_dict.get(x,["q_proj"])



  # enable quantized training

  embedding_model = prepare_model_for_kbit_training(embedding_model)

  config = LoraConfig(

    r=8,

    lora_alpha=32,

    # target_modules=["k_proj"],

    target_modules = target_modules,#TODOOOOOOOOOOOOO

    lora_dropout=0.1,

    bias="none",

    task_type="SEQ_CLS",

  )

  # LoRA trainable version of model

  embedding_model = get_peft_model(embedding_model, config)



  # trainable parameter count

  embedding_model.print_trainable_parameters()

device="cuda" if torch.cuda.is_available() else 'cpu'

In [179]:
device

'cuda'

In [180]:
embedding_model = embedding_model.to(device)

for param in embedding_model.parameters():
    param.data = param.data.contiguous()


In [181]:

# Custom Dataset for batching
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize a single example
        return self.texts[idx]
        # return self.tokenizer(
        #     self.texts[idx],
        #     truncation=True,
        #     padding="max_length",
        #     max_length=self.max_length,
        #     return_tensors="pt"
        # )


In [182]:

# Batch extraction function
def extract_embeddings(texts, model, batch_size=32):
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    embeddings = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting embeddings"):
            # Move batch to device
            # input_ids = torch.Tensor(batch['input_ids']).squeeze().to(device)
            # attention_mask = torch.Tensor(batch['attention_mask']).squeeze().to(device)

            # Forward pass through the model
            cls_embeddings = model.encode(batch)
            embeddings.append(cls_embeddings)

    return np.vstack(embeddings)

In [183]:
# Extract embeddings for train and validation datasets
x_train_embeddings = extract_embeddings(x_train, embedding_model, batch_size=32)
x_valid_embeddings = extract_embeddings(x_valid, embedding_model, batch_size=32)

Extracting embeddings: 100%|██████████| 24/24 [00:13<00:00,  1.76it/s]


In [184]:
# Flatten labels for XGBoost (multi-label classification)
y_train_flat = np.array(y_train)
y_valid_flat = np.array(y_valid)

In [185]:
from sklearn.svm import SVC

In [186]:
svm_models = {}
predictions = {}

for i, label in enumerate(label_columns):
    print(f"Training SVM for label: {label}")

    # Use the computed class weights for balancing the classes
    class_weight = {0: 1, 1: class_weights_tensor[i].item()}  # Adjust the class weights

    svm_model = SVC(class_weight={0: 1, 1: class_weights_tensor[i].item()})

    svm_model.fit(x_train_embeddings, y_train_flat[:, i])
    svm_models[label] = svm_model

    # Predict on validation data
    predictions[label] = svm_model.predict(x_valid_embeddings)

Training SVM for label: anger
Training SVM for label: disgust
Training SVM for label: fear
Training SVM for label: joy
Training SVM for label: sadness
Training SVM for label: surprise


In [ ]:
# Evaluate the models
results = []
for i, label in enumerate(label_columns):
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_valid_flat[:, i],
        predictions[label],
        average="binary"
    )
    accuracy = accuracy_score(y_valid_flat[:, i], predictions[label])
    results.append({
        "Label": label,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "Accuracy": accuracy
    })

In [188]:
# Display evaluation results
results_df = pd.DataFrame(results)
print(results_df)

      Label  Precision    Recall  F1 Score  Accuracy
0     anger   0.524096  0.595890  0.557692  0.817219
1   disgust   0.400000  0.125000  0.190476  0.954967
2      fear   0.769231  0.178571  0.289855  0.935099
3       joy   0.680328  0.610294  0.643411  0.878146
4   sadness   0.611650  0.547826  0.577982  0.878146
5  surprise   0.777778  0.122807  0.212121  0.931126


In [189]:
print(f"{x} evaluates:")

for k, res in enumerate(results):

  print(f"Class '{res['Label']}': Precision: {res['Precision']:.4f}, Recall: {res['Recall']:.4f}, F1: {res['F1 Score']:.4f}")

print(f"Overall accuracy: {accuracy:.4f}")

print("-------------------------------------------------------------------------\n")

rufimelo/bert-large-portuguese-cased-sts evaluates:
Class 'anger': Precision: 0.5241, Recall: 0.5959, F1: 0.5577
Class 'disgust': Precision: 0.4000, Recall: 0.1250, F1: 0.1905
Class 'fear': Precision: 0.7692, Recall: 0.1786, F1: 0.2899
Class 'joy': Precision: 0.6803, Recall: 0.6103, F1: 0.6434
Class 'sadness': Precision: 0.6117, Recall: 0.5478, F1: 0.5780
Class 'surprise': Precision: 0.7778, Recall: 0.1228, F1: 0.2121
Overall accuracy: 0.9311
-------------------------------------------------------------------------



In [190]:
# Initialize a DataFrame to store the evaluation results for each language
lang_metrics_df = pd.DataFrame(columns=["Language", "Precision", "Recall", "F1_score", "Accuracy"])

# Iterate over each language
for lang in langs:
    # Filter the validation data for the current language
    lang_valid_data = valid[valid['lang'] == lang]
    lang_x_valid = lang_valid_data['clean_message'].values.tolist()
    lang_y_valid = lang_valid_data[label_columns].values.tolist()

    if len(lang_valid_data) == 0:
        continue

    # Extract embeddings for the current language's validation data
    lang_x_valid_embeddings = extract_embeddings(lang_x_valid, embedding_model)
    print()
    # Initialize metrics storage
    all_precisions, all_recalls, all_f1_scores, all_accuracies = [], [], [], []

    # Evaluate each label separately
    for i, label in enumerate(label_columns):
        # Predict using the corresponding svm model
        lang_predictions = svm_models[label].predict(lang_x_valid_embeddings)

        # Compute metrics
        precision, recall, f1, _ = precision_recall_fscore_support(
            np.array(lang_y_valid)[:, i],  # True labels for this label
            lang_predictions,
            average="binary",
            zero_division=0
        )
        accuracy = accuracy_score(np.array(lang_y_valid)[:, i], lang_predictions)

        # Store metrics
        all_precisions.append(precision)
        all_recalls.append(recall)
        all_f1_scores.append(f1)
        all_accuracies.append(accuracy)

    # Aggregate metrics (average across all labels for the language)
    new_data = pd.DataFrame({
        "Language": [lang],
        "Precision": [np.mean(all_precisions)],
        "Recall": [np.mean(all_recalls)],
        "F1_score": [np.mean(all_f1_scores)],
        "Accuracy": [np.mean(all_accuracies)]
    })

    lang_metrics_df = pd.concat([lang_metrics_df, new_data], ignore_index=True)

# Display the evaluation results for each language
print(lang_metrics_df)


Extracting embeddings: 100%|██████████| 15/15 [00:09<00:00,  1.59it/s]


<ipython-input-190-9b6d3eac4dcb>:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lang_metrics_df = pd.concat([lang_metrics_df, new_data], ignore_index=True)
Extracting embeddings: 100%|██████████| 10/10 [00:02<00:00,  4.51it/s]



  Language  Precision    Recall  F1_score  Accuracy
0     ptbr   0.569083  0.363373  0.387638  0.887298
1     ptmz   0.656838  0.269040  0.338765  0.916944


In [ ]:
import pickle
# Save the svm models if needed
for label, model in svm_models.items():
    with open(f"svm_model_{label}.pkl", 'wb') as f:
      pickle.dump(model, f)

In [192]:
!zip svm_models_pkl.zip svm_model_*.pkl

updating: svm_model_anger.pkl (deflated 75%)
updating: svm_model_disgust.pkl (deflated 75%)
updating: svm_model_fear.pkl (deflated 75%)
updating: svm_model_joy.pkl (deflated 75%)
updating: svm_model_sadness.pkl (deflated 75%)
updating: svm_model_surprise.pkl (deflated 75%)
